<a href="https://colab.research.google.com/github/everardogarciaf/portfolio/blob/main/Everardo_Garcia_Flores_senior_data_analyst_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Senior Analyst SQL Homework

You will be working with a fictious ecommerce dataset provided by Looker, hosted publicly on BigQuery https://console.cloud.google.com/marketplace/product/bigquery-public-data/thelook-ecommerce

Click this link for a sample query against this data set
https://console.cloud.google.com/bigquery?sq=258440106949:bfb3c07d98e6487ca028cd23ea86d361


You can use the Explorer as your scratch, but will write your final queries in this notebook and save the code and the results when you are done and send the link to your saved result back to data_team@cypress.io. 

You will write some SQL to return some specific summaries, then you will perform a free response analysis from a prompt. Spend no more than a few hours total completing this and email immediately when you're done

# Example Queries

In [2]:
# first you must authenticate with your Google account to query against the project you set up
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [8]:
# prettier table display
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [ ]:
%%bigquery --project cypress-hw

# replace 'your-project-id' with your personal project id where you linked the data set

# result of query below is returned immediately
SELECT 
  * 
FROM `bigquery-public-data.thelook_ecommerce.products` limit 10

In [8]:
%%bigquery df --project cypress-hw
# replace 'your-project-id' with your personal project id where you linked the data set

#result of same query below is returned to memory as the variable df, which is a pandas dataframe object
SELECT 
  * 
FROM `bigquery-public-data.thelook_ecommerce.products` limit 10

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
#---------TEST AREA---------

%%bigquery --project cypress-hw
# replace 'your-project-id' with your personal project id where you linked the data set

----------Do some exploratory work on distribution centers
---Location
---Order Frequency
---Monthly Inventory
---

SELECT
  i.id AS inventory_item_id
  , i.product_id
  , i.created_at
  , i.sold_at
  , i.cost
  , d.id AS distribution_center_id
  , d.name AS dist_center_location
  , CASE WHEN i.sold_at >= i.created_at THEN 0 ELSE 1 END AS is_in_warehouse

FROM bigquery-public-data.thelook_ecommerce.inventory_items i
LEFT JOIN bigquery-public-data.thelook_ecommerce.distribution_centers d
  ON d.id = i.product_distribution_center_id
LIMIT 3000


Query is running:   0%|          |

Downloading:   0%|          |

,inventory_item_id,product_id,created_at,sold_at,cost,distribution_center_id,dist_center_location,is_in_warehouse
0,71793,16898,2022-09-09 19:30:58+00:00,2022-10-29 16:44:58+00:00,13.925,3,Houston TX,0
1,71794,16898,2023-01-07 15:16:00+00:00,NaT,13.925,3,Houston TX,1
2,71795,16898,2021-11-18 09:58:00+00:00,NaT,13.925,3,Houston TX,1
3,71796,16898,2022-08-23 08:25:00+00:00,NaT,13.925,3,Houston TX,1
4,153052,16898,2022-01-29 17:21:48+00:00,2022-02-27 13:31:48+00:00,13.925,3,Houston TX,0
...,...,...,...,...,...,...,...,...
2995,446956,6897,2022-07-15 08:29:00+00:00,2022-08-04 12:06:00+00:00,18.440,4,Los Angeles CA,0
2996,446957,6897,2022-07-12 15:37:00+00:00,NaT,18.440,4,Los Angeles CA,1
2997,446958,6897,2021-06-16 16:19:00+00:00,NaT,18.440,4,Los Angeles CA,1
2998,446959,6897,2021-12-24 01:23:00+00:00,NaT,18.440,4,Los Angeles CA,1


In [ ]:
df

#Query Assignment 1

Your partner in the marketing department wants a list of past customers to send a promotion. 
Fill in and run the cell below with a query that returns all user ids, first and last names and emails, who have ordered at least $20 lifetime, total over all their orders (go to the Explorer to browse the schema to find the necessary tables)

In [1]:
%%bigquery --project cypress-hw
# replace 'your-project-id' with your personal project id where you linked the data set

--Pull the total sale price for each order
# WITH order_totals AS(
SELECT
    o.order_id
    , o.user_id
    , count(o.order_id) AS num_of_items
    # , sum(oi.sale_price) AS order_total
FROM bigquery-public-data.thelook_ecommerce.orders o
# LEFT JOIN bigquery-public-data.thelook_ecommerce.order_items oi
#     ON oi.order_id = o.order_id
# GROUP BY 1, 2
# )

--Find all users and their details, then get the sum of the total on all that user's orders
--for users with a lifetime order total >= $20
# SELECT
#     u.id AS user_id
#     , u.first_name
#     , u.last_name
#     , u.email
#     , count(DISTINCT ot.order_id) AS num_of_orders
#     , sum(order_total) AS lifetime_order_total
# FROM bigquery-public-data.thelook_ecommerce.users u
# LEFT JOIN order_totals ot
#     ON ot.user_id = u.id
# GROUP BY 1, 2, 3, 4
# HAVING lifetime_order_total >= 20
# ORDER BY 1


ERROR:
 ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7f7e6b2bfca0>)


#Query Assignment 2

Your partner in finance and operations wants to know total profit last month (Nov 2022) by distribution center, to see if any of the distributions centers are performing much better than others. Total profit is the sale price of items sold from orders (ignore unsold inventory) minus the product cost of sold items found in the inventory table (for simplicity assume everything sold at full retail price, no discounts, returns or shrink). For extra credit, provide the month over month diff in the profit margin, from Oct 2022 to Nov 2022. 

In [84]:
%%bigquery --project cypress-hw
# replace 'your-project-id' with your personal project id where you linked the data set


--Find each ordered item and calculate its profit (sale_price - product_cost)
--Assume the date on which the order was created is the finance order date
WITH all_ordered_items AS(
SELECT DISTINCT
    oi.id as order_item_id
    , o.order_id
    , o.user_id
    , i.product_id
    , i.product_distribution_center_id as dist_center_id
    , date_trunc(CAST(o.created_at AS DATE), MONTH) AS order_month
    , oi.sale_price
    , i.cost
    , oi.sale_price - i.cost AS item_profit
FROM bigquery-public-data.thelook_ecommerce.orders o
LEFT JOIN bigquery-public-data.thelook_ecommerce.order_items oi
    ON oi.order_id = o.order_id
LEFT JOIN bigquery-public-data.thelook_ecommerce.inventory_items i
    ON i.product_id = oi.product_id
# WHERE o.order_id = 8445
LIMIT 3000
)

--Find the monthly profits by distribution center
, monthly_profits AS (
SELECT
  a.order_month
  , a.dist_center_id
  , d.name
  , sum(a.item_profit) as total_profit
  , count(DISTINCT a.order_id) AS number_of_orders
FROM all_ordered_items a
LEFT JOIN bigquery-public-data.thelook_ecommerce.distribution_centers d
    ON d.id = a.dist_center_id
WHERE order_month BETWEEN '2022-06-01' AND '2022-11-30'
GROUP BY 1, 2, 3
ORDER BY 2, 1, total_profit DESC
)

--Find the monthly profits from the previous month
, previous_month_profit AS (
SELECT
    m.dist_center_id
    , m.name AS dist_center_location
    , m.order_month
    , m.total_profit AS monthly_profit
    , LAG(m.total_profit, 1, 0) OVER(PARTITION BY m.dist_center_id ORDER BY m.dist_center_id, m.order_month) AS previous_month_profit
    , SUM(number_of_orders) AS total_orders
FROM monthly_profits m
GROUP BY 1, 2, 3, 4
ORDER BY 1
)

SELECT *
  , monthly_profit - previous_month_profit AS month_over_month_profit_diff
  , monthly_profit / total_orders AS avg_order_profit
FROM previous_month_profit p
WHERE order_month = '2022-11-01'

Query is running:   0%|          |

Downloading:   0%|          |

,dist_center_id,dist_center_location,order_month,monthly_profit,previous_month_profit,total_orders,month_over_month_profit_diff,avg_order_profit
0,1,Memphis TN,2022-11-01,1092.598060,864.956187,21,227.641873,52.028479
1,2,Chicago IL,2022-11-01,986.346087,769.321859,36,217.024227,27.398502
2,3,Houston TX,2022-11-01,1408.914041,609.177191,32,799.736850,44.028564
3,4,Los Angeles CA,2022-11-01,787.465911,561.448463,21,226.017448,37.498377
4,5,New Orleans LA,2022-11-01,292.160153,302.922970,13,-10.762817,22.473858
5,6,Port Authority of New York/New Jersey NY/NJ,2022-11-01,562.841572,510.117021,14,52.724551,40.202969
6,7,Philadelphia PA,2022-11-01,587.021405,362.762773,19,224.258633,30.895863
7,8,Mobile AL,2022-11-01,676.725314,798.561549,22,-121.836236,30.760242
8,9,Charleston SC,2022-11-01,356.985510,299.605261,15,57.380250,23.799034
9,10,Savannah GA,2022-11-01,244.404731,635.752081,15,-391.347350,16.293649


#Free Response Assignment

After looking at the profit margins of distribution centers, the VP of finance is very impressed with your skills and asks you look into the data more and see if you can find any more efficiencies for the distribution centers. Using your analytical best judgement, write one or more additional queries and show what you found. This should be doable 100% with SQL, but if you know Python or R for a chart you can use it in this section (this doesn't need machine learning or advanced stats techniques at all, this is pure exploratory mining). A few ideas to get started - how overstocked or understocked are they? is inventory well allocated across centers or not? Write some comments on any recommendations from your findings

In [ ]:
%%bigquery --project your-project-id

----------Do some exploratory work on distribution centers
---Location
---Order Frequency
---Monthly Inventory
---




Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,replace this
